# VDMA scrapping

They do not use different links for each page, instead they use AJAX to query the results for each page. This problem is solved with the code

This is the website: https://www.vdma.org/members

In [119]:
#importing libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

#Defining the bas url
base_url = "https://www.vdma.org/mitglieder?p_p_id=org_vdma_publicusers_portlet_PublicUsersPortlet_INSTANCE_H0VO3QljCiRM&p_p_lifecycle=2&p_p_state=normal&p_p_mode=view&p_p_resource_id=getPage&p_p_cacheability=cacheLevelPage"

# Initial parameters
searchQuery = ""
letters = ""
items_all_pages =[]

#This will be used in case the request breaks, which happens every now and then 
sleep_n = 3


#Defining get attributes function
def get_attributes(response):
    data = json.loads(response.text)
    publicUserList = json.loads(data['publicUserList'])
    content = publicUserList['content']
    attributes = ['companyName', 'name2', 'name3', 'address', 'city', 'country', 'phoneNum', 'email', 'webAddr']
    items = []
    for item in content:
        items.append(item)
    return items

#Loop through all of the pages of the website
for page in range(1,364):
    params = {
            '_org_vdma_publicusers_portlet_PublicUsersPortlet_INSTANCE_H0VO3QljCiRM_query': searchQuery,
            '_org_vdma_publicusers_portlet_PublicUsersPortlet_INSTANCE_H0VO3QljCiRM_s': letters,
            '_org_vdma_publicusers_portlet_PublicUsersPortlet_INSTANCE_H0VO3QljCiRM_page': page
    }

    response = requests.post(base_url, data=params)
    #If response is null, wait, try again, until it works. 
    while json.loads(response.text)['publicUserList']=='null':
        time.sleep(sleep_n)
        response = requests.post(base_url, data=params)
        sleep_n+=1
    sleep_n = 3
    items_page = get_attributes(response)
    items_all_pages.append(items_page)
    
#Flatten list of list of dicts
flattened_list = [element for sublist in items_all_pages for element in sublist]

#Create and save dataframe
df = pd.DataFrame(flattened_list)
df.to_csv('accounts.csv')

KeyboardInterrupt: 